# Immigration Activity of the People of Ecuador
### Data Engineering Capstone Project

#### Project Summary

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd, numpy as np

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

from pyspark.sql.functions import udf, col, monotonically_increasing_id, isnan, when, count, avg
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, date_format,to_date,to_timestamp
from pyspark.sql.types import * 

import re

### Step 1: Scope the Project and Gather Data

#### Scope

In this project, we will aggregate I94 immigration data by destination city to form our first dimension table. Next we will aggregate city temperature data by city to form the second dimension table. The two datasets will be joined on destination city to form the fact table. The final database is optimized to query on immigration events to determine if temperature affects the selection of destination cities. We will also take a particular look at the immigration activity of the Ecuadorians. Spark will be used to process the data.

#### Describe and Gather Data

The I94 immigration data comes from the US National Tourism and Trade Office. It is provided in SAS7BDAT format which is a binary database storage format. Some relevant attributes include:
- i94yr = 4 digit year
- i94mon = numeric month
- i94cit = 3 digit code of birth country
- i94res = 3 digit code of residence country
- i94port = 3 character code of destination USA city
- arrdate = arrival date in the USA
- i94mode = 1 digit travel code
- i94addr = 2 letter code of US states
- depdate = departure date from the USA
- i94visa = reason for immigration

The temperature data comes from Kaggle ([link](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)). It is provided in csv format. Some relevant attributes include:
- AverageTemperature = average temperature (in Celsius)
- City = city name
- Country = country name
- Latitude= latitude
- Longitude = longitude

The US demographics data comes from Opendatasoft ([link](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)). It is provided in csv format. Some relevant attributes include:
- City = city name
- State = state name
- Total Population = total city population 
- Race = selection of White, Hispanic or Latino, Asian, Black or African-American, American Indian and Alaska Native
- Count = Number Count based on respective Race selection

In [2]:
# Read in April 2016 I94 immigration data via Spark
conf = SparkConf().setMaster("local[*]").set("spark.network.timeout", "600s")

spark = SparkSession \
    .builder \
    .appName("Capstone Project") \
    .config(conf=conf) \
    .getOrCreate()

i94_data = spark.read.parquet("sas_data/")

In [3]:
# Use Spark for data exploration
i94_data.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,...,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,...,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,...,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,None,M,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1


In [4]:
# Checking how many rows
i94_data.count()

3096313

In [5]:
# Read in the temperature data into Pandas for exploration
file = 'GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(file, sep=',', nrows=5)

In [6]:
#Display first five entries of temperature data
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


## Step 2: Clean the Data

For the I94 immigration data, we want to drop all entries where the destination city code i94port is not a valid value (e.g., XXX, 99, etc) as described in I94_SAS_Labels_Description.SAS. For the temperature data, we want to drop all entries where AverageTemperature is Null value, then take the average yearly April temperature for each location and lastly add the i94port of the location in each entry. We would also change some of the columns from string to their appropriate data type. 
For the US demographics data, we extract the columns that we will need to perform our data analysis. 

In [7]:
# Cleaning the immigration data

# Create a dictionary of valid i94 codes 
re_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
i94port_valid = {}
with open('i94port_valid.txt') as f:
     for line in f:
         match = re_obj.search(line)
         i94port_valid[match[1]]=[match[2]]
            
def clean_i94_data(data):
    '''
    
    Description: This function filter out entries where i94port codes are not valid
    Arguments:
        data: a Spark dataframe    
    '''
    
    data = data.filter(data.i94port.isin(list(i94port_valid.keys())))

    return data

In [8]:
# Test clean_i94_data function
i94_data_test = spark.read.parquet("sas_data/")

df_i94_test = clean_i94_data(i94_data_test)

df_i94_test.select(df_i94_test.i94port).show(n=20)

+-------+
|i94port|
+-------+
|    LOS|
|    LOS|
|    LOS|
|    LOS|
|    LOS|
|    HHW|
|    HHW|
|    HHW|
|    HOU|
|    LOS|
|    NEW|
|    LOS|
|    WAS|
|    LOS|
|    LOS|
|    MIA|
|    SFR|
|    HOU|
|    HOU|
|    LOS|
+-------+
only showing top 20 rows



In [9]:
## Clean i94port
df_i94_test.filter(df_i94_test.i94port == 'NaN').count()

0

In [10]:
# Clean temperature data
df_temp = spark.read.format("csv").option("header","true").load(file)

# Filter out entries with NaN average temperature
df_temp = df_temp.filter(df_temp.AverageTemperature != 'NaN')

# Change data type of dt from string to date
df_temp = df_temp.withColumn("dt", col("dt").cast("date"))


# Change data type of Average Temperature from string to double
df_temp = df_temp.withColumn("AverageTemperature",col("AverageTemperature").cast("Double"))


# Extract average temperature for the month of April for every city and country
df_temp = df_temp.withColumn("Month", month("dt"))
df_temp = df_temp.filter(col("Month") == 4)\
                 .groupBy("City", "Country", "Latitude", "Longitude") \
                 .agg(avg("AverageTemperature").alias("AverageTemperature"))

# Remove duplicate locations
# df_temp = df_temp.dropDuplicates(['City', 'Country'])

@udf()
def get_i94port(city):
    '''
    Descriptions: This function gets the corresponding i94 port code for the respective city
    Arguments:
        city: city name
    '''
    
    for key in i94port_valid:
        if city.lower() in i94port_valid[key][0].lower():
            return key
        
# Add iport94 code based on city name
df_temp = df_temp.withColumn("i94port", get_i94port(df_temp.City))

# Remove entries with iport94 is null
df_temp = df_temp.filter(df_temp.i94port != 'null')

In [11]:
# Show results
df_temp.show()

+-----------+--------------------+--------+---------+------------------+-------+
|       City|             Country|Latitude|Longitude|AverageTemperature|i94port|
+-----------+--------------------+--------+---------+------------------+-------+
|       Aden|               Yemen|  13.66N|   45.41E| 26.58692753623188|    SRQ|
|   Columbus|       United States|  32.95N|   85.21W|16.635823076923067|    COL|
|    Durango|              Mexico|  24.92N|  104.63W|18.936804469273742|    DRO|
|  Melbourne|           Australia|  37.78S|  144.41E|13.714534883720939|    MLB|
|      Derby|      United Kingdom|  53.84N|    1.36W| 7.290939849624056|    DER|
|   Montreal|              Canada|  45.81N|   72.69W|3.9470988593155876|    MON|
|       Nice|              France|  44.20N|    6.71E| 7.833003759398493|    CND|
| Washington|       United States|  39.38N|   76.99W|11.250669230769233|    WAS|
|   Hamilton|              Canada|  42.59N|   80.73W| 6.224026615969584|    HAM|
|   Veracruz|              M

#### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model

The first dimension table will contain events from the I94 immigration data. The columns below will be extracted from the immigration dataframe:
- i94yr = 4 digit year
- i94mon = numeric month
- i94cit = 3 digit code of birth country
- i94res = 3 digit code of residence country
- i94port = 3 character code of destination USA city
- arrdate = arrival date in the USA
- i94mode = 1 digit travel code
- i94addr = 2 letter code of US states
- depdate = departure date from the USA
- i94visa = reason for immigration

The second dimension table will contain city temperature data. The columns below will be extracted from the temperature dataframe:
- i94port = 3 character code of destination city (mapped from immigration data during cleanup step)
- AverageTemperature = average temperature
- City = city name
- Country = country name
- Latitude= latitude
- Longitude = longitude

The third dimension table will contain US demographics data. The columns below will be extracted from the us demographics dataframe:
- City = city name
- State = state name
- Total Population = total city population 
- Race = selection of White, Hispanic or Latino, Asian, Black or African-American, American Indian and Alaska Native
- Count = Number Count based on respective Race selection


The final table will contain information from the I94 immigration data joined with the city temperature data on i94port:
- i94yr = 4 digit year
- i94mon = numeric month
- i94res = 3 digit code of residence country
- i94cit = 3 digit code of birth country
- i94port = 3 character code of destination city
- arrdate = arrival date
- destination_city = full name of city
- depdate = departure date
- i94visa = reason for immigration
- hispanic_ratio = percentage of Hispanic or Latino people in the city
- AverageTemperature = average temperature of destination city

The tables will be saved to Parquet files partitioned by city (i94port).

#### 3.2 Mapping Out Data Pipelines

The pipeline steps are described below:
1. Clean I94 data as described in step 2 to create Spark dataframe df_immigration
2. Clean temperature data as described in step 2 to create Spark dataframe df_temp (already performed)
3. Clean demographics data as described in step 2 to create Spark dataframe df_demographics 
4. Create immigration dimension table by selecting relevant columns from df_immigration and write to parquet file partitioned by i94port
5. Create temperature dimension table by selecting relevant columns from df_temp and write to parquet file partitioned by i94port
6. Create demographics dimension table by selecting relevant columns from df_temp and write to parquet file partitioned by i94port
7. Create final table by joining immigration, temperature, demographics dimension tables on i94port and write to parquet file partitioned by i94port

#### Step 4: Run Pipelines to Model the Data

4.1 Create the data model

Build the data pipelines to create the data model.

In [12]:
# Obtain immigration data
immigration_data = spark.read.parquet("sas_data/")

# Clean i94 immigration data
df_immigration = clean_i94_data(immigration_data)

# Extract columns for immigration dimension table
immigration_table = df_immigration.select(
                                        df_immigration.i94yr.alias("i94yr"),
                                        df_immigration.i94mon.alias("i94mon"),
                                        df_immigration.i94cit.alias("i94cit"),
                                        df_immigration.i94res.alias("i94res"),
                                        df_immigration.i94port.alias("i94port"),
                                        df_immigration.arrdate.alias("arrdate"),
                                        df_immigration.i94mode.alias("i94mode"),
                                        df_immigration.i94addr.alias("i94addr"),
                                        df_immigration.depdate.alias("depdate"),
                                        df_immigration.i94visa.alias("i94visa")
                                    ).filter(df_immigration.i94addr != 'NaN')

In [13]:
# Write immigration dimension table to parquet files partitioned by i94port
immigration_table.write.mode("overwrite").partitionBy("i94port").parquet("/results/immigration.parquet")

In [16]:
immigration_table.show()

+------+------+------+------+-------+-------+-------+-------+-------+-------+
| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94visa|
+------+------+------+------+-------+-------+-------+-------+-------+-------+
|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|    1.0|
|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|    1.0|
|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     WA|20582.0|    1.0|
|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     WA|20588.0|    1.0|
|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     WA|20588.0|    1.0|
|2016.0|   4.0| 245.0| 464.0|    HHW|20574.0|    1.0|     HI|20579.0|    2.0|
|2016.0|   4.0| 245.0| 464.0|    HHW|20574.0|    1.0|     HI|20586.0|    2.0|
|2016.0|   4.0| 245.0| 464.0|    HHW|20574.0|    1.0|     HI|20586.0|    2.0|
|2016.0|   4.0| 245.0| 464.0|    HOU|20574.0|    1.0|     FL|20581.0|    2.0|
|2016.0|   4.0| 245.0| 464.0|    LOS|20574.0|    1.0|     CA|205

In [17]:
# Extract & arrange columns for temperature dimension table
temp_table = df_temp.select(["i94port", "City", "Country", "Latitude", "Longitude", "AverageTemperature"])

In [ ]:
# Write temperature dimension table to parquet files paritioned by i94port
temp_table.write.mode("overwrite").partitionBy("i94port").parquet("/results/temperature.parquet")

In [19]:
temp_table.show()

+-------+-----------+--------------------+--------+---------+------------------+
|i94port|       City|             Country|Latitude|Longitude|AverageTemperature|
+-------+-----------+--------------------+--------+---------+------------------+
|    SRQ|       Aden|               Yemen|  13.66N|   45.41E| 26.58692753623188|
|    COL|   Columbus|       United States|  32.95N|   85.21W|16.635823076923067|
|    DRO|    Durango|              Mexico|  24.92N|  104.63W|18.936804469273742|
|    MLB|  Melbourne|           Australia|  37.78S|  144.41E|13.714534883720939|
|    DER|      Derby|      United Kingdom|  53.84N|    1.36W| 7.290939849624056|
|    MON|   Montreal|              Canada|  45.81N|   72.69W|3.9470988593155876|
|    CND|       Nice|              France|  44.20N|    6.71E| 7.833003759398493|
|    WAS| Washington|       United States|  39.38N|   76.99W|11.250669230769233|
|    HAM|   Hamilton|              Canada|  42.59N|   80.73W| 6.224026615969584|
|    VER|   Veracruz|       

In [20]:
demo = 'us-cities-demographics.csv'
df_demographics = spark.read.csv(demo, header='true', sep=";")
print('df_demographics ', df_demographics.count())

df_demographics  2891


In [21]:
# Extract appropriate columans and add iport94 code based on city name
# And in addition, dropping null rows for i94port

df_demographics = df_demographics.select(["City", "State", "Total Population", "Race", "Count"])
df_demographics = df_demographics.withColumn("i94port", get_i94port(df_demographics.City))
df_demographics = df_demographics.filter(df_demographics.i94port != 'NaN')

In [22]:
df_demographics.limit(10).toPandas()

,City,State,Total Population,Race,Count,i94port
0,Newark,New Jersey,281913,White,76402,NEW
1,Peoria,Illinois,118661,American Indian and Alaska Native,1343,PIA
2,Philadelphia,Pennsylvania,1567442,Asian,122721,PHI
3,Fort Myers,Florida,74015,White,50169,FMY
4,Laredo,Texas,255789,American Indian and Alaska Native,1253,LCB
5,Allen,Pennsylvania,120207,Black or African-American,22304,MCA
6,New Haven,Connecticut,130310,American Indian and Alaska Native,2205,NWH
7,Salt Lake City,Utah,192660,Asian,13153,SLC
8,Suffolk,Virginia,88161,Black or African-American,39107,FOK
9,Los Angeles,California,3971896,White,2177650,LOS


In [23]:
# Create temporary views of the immigration, temperature, and demographics data
df_immigration.createOrReplaceTempView("immigration_view")
df_temp.createOrReplaceTempView("temp_view")
df_demographics.createOrReplaceTempView("demogr_view")

In [24]:
# Create the final table by joining the immigration, temperature, and demographics views
fact_table = spark.sql('''
    SELECT immigration_view.i94yr as year,
            immigration_view.i94mon as month,
            immigration_view.i94cit as birth_country,
            immigration_view.i94res as residence_country,
            immigration_view.i94port as i94port,
            immigration_view.arrdate as arrival_date,
            temp_view.City as destination_city,
            immigration_view.depdate as departure_date,
            immigration_view.i94visa as visa_type,
            (demogr_view.Count / demogr_view.`Total Population`) as hispanic_ratio,
            temp_view.AverageTemperature as temperature

FROM immigration_view
    JOIN temp_view ON (immigration_view.i94port = temp_view.i94port)
    JOIN demogr_view ON (immigration_view.i94port = demogr_view.i94port)
    WHERE demogr_view.Race = 'Hispanic or Latino'
''')

In [ ]:
# Write fact table to parquet files paritioned by i94port
fact_table.write.mode("overwrite").partitionBy("i94port").parquet("/results/fact.parquet")

In [26]:
fact_table.limit(10).toPandas()

,year,month,birth_country,residence_country,i94port,arrival_date,destination_city,departure_date,visa_type,hispanic_ratio,temperature
0,2016.0,4.0,135.0,135.0,SRQ,20561.0,Aden,20580.0,1.0,0.223965,26.586928
1,2016.0,4.0,689.0,689.0,SRQ,20560.0,Aden,20567.0,2.0,0.223965,26.586928
2,2016.0,4.0,311.0,353.0,SRQ,20560.0,Aden,20571.0,2.0,0.223965,26.586928
3,2016.0,4.0,298.0,298.0,SRQ,20560.0,Aden,20629.0,2.0,0.223965,26.586928
4,2016.0,4.0,298.0,298.0,SRQ,20560.0,Aden,20578.0,2.0,0.223965,26.586928
5,2016.0,4.0,298.0,298.0,SRQ,20560.0,Aden,20578.0,2.0,0.223965,26.586928
6,2016.0,4.0,296.0,296.0,SRQ,20560.0,Aden,20579.0,2.0,0.223965,26.586928
7,2016.0,4.0,296.0,296.0,SRQ,20560.0,Aden,20579.0,2.0,0.223965,26.586928
8,2016.0,4.0,296.0,296.0,SRQ,20560.0,Aden,20574.0,1.0,0.223965,26.586928
9,2016.0,4.0,296.0,296.0,SRQ,20560.0,Aden,20571.0,2.0,0.223965,26.586928


#### 4.2 Data Quality Checks
Run Quality Checks

In [27]:
def quality_check(df, description):
    '''
    Description: This function performs data quality checks to ensure that the dataframe consists of rows 
    Arguments:
        df: Spark dataframe
        description: brief description of the dataframe
    '''
    result = df.count()
    if result == 0:
        print("Data quality check failed for {} with zero records".format(description))
    else:
        print("Data quality check passed for {} with {} records.".format(description, result))

In [ ]:
# Perform data quality check
quality_check(df_demographics, "us_demographic table")
quality_check(df_immigration, "immigration table")
quality_check(df_temp, "temperature table")

Data quality check passed for us_demographic table with 878 records.
Data quality check passed for immigration table with 3092066 records.


In [30]:
# Filtering the table to show data related to Ecuador. The code for Ecucador is 692

ecuador_data = fact_table.filter(("birth_country = 692.0 OR residence_country = 692.0")).toPandas()  
ecuador_data.shape 

(53443, 11)

In [31]:
ecuador_data.head()

,year,month,birth_country,residence_country,i94port,arrival_date,destination_city,departure_date,visa_type,hispanic_ratio,temperature
0,2016.0,4.0,692.0,692.0,SRQ,20563.0,Aden,20578.0,2.0,0.223965,26.586928
1,2016.0,4.0,692.0,692.0,SRQ,20563.0,Aden,20573.0,2.0,0.223965,26.586928
2,2016.0,4.0,692.0,692.0,SRQ,20563.0,Aden,20573.0,2.0,0.223965,26.586928
3,2016.0,4.0,692.0,692.0,SRQ,20563.0,Aden,20570.0,2.0,0.223965,26.586928
4,2016.0,4.0,692.0,692.0,SRQ,20563.0,Aden,20570.0,2.0,0.223965,26.586928


In [32]:
ecuador_data['count'] = 1

In [33]:
ecuador_data.groupby('i94port').agg({'count':np.size, 'hispanic_ratio': np.mean, 'temperature': np.mean})\
                                .sort_values('count', ascending=False).reset_index().head(10)

,i94port,count,hispanic_ratio,temperature
0,MIA,14908,0.725510,22.596529
1,NYC,14830,0.290644,7.966164
2,LOS,6330,0.487609,18.309527
3,FTL,4513,0.157615,22.596529
4,ORL,4426,0.329643,21.882888
5,NEW,1622,0.356252,16.548255
6,ATL,1184,0.040211,14.354050
7,CHI,1150,0.288810,9.288686
8,HOU,863,0.447288,20.302892
9,SFR,545,0.152765,13.638939


In [34]:
# Finding out the average April temperature in Ecuador
ecuador_temp = df_temp.filter(col("Country") == "Ecuador").show()

+-----+-------+--------+---------+------------------+-------+
| City|Country|Latitude|Longitude|AverageTemperature|i94port|
+-----+-------+--------+---------+------------------+-------+
|Quito|Ecuador|   0.80S|   77.95W| 16.85527142857142|    UIO|
+-----+-------+--------+---------+------------------+-------+



### Conclusion
We learned that the most popular city for Ecuardorians to immigrate to in April 2016 is Miami, followed by New York City (NYC). It’s important to note that NYC’s temperature is far below the average temperature for Ecuador and hispanic ratio is about a third of the population. Looking at the top ten cities, the temperature and the hispanic ratio does not seem to play a major role in city selection.

In [ ]:
# close spark session
spark.stop()

#### 4.3 Data dictionary

The first dimension table will contain events from the I94 immigration data. The columns below will be extracted from the immigration dataframe:
- i94yr = 4 digit year
- i94mon = numeric month
- i94cit = 3 digit code of birth country
- i94res = 3 digit code of residence country
- i94port = 3 character code of destination USA city
- arrdate = arrival date in the USA
- i94mode = 1 digit travel code
- i94addr = 2 letter code of US states
- depdate = departure date from the USA
- i94visa = reason for immigration

The second dimension table will contain city temperature data. The columns below will be extracted from the temperature dataframe:
- i94port = 3 character code of destination city (mapped from immigration data during cleanup step)
- AverageTemperature = average temperature
- City = city name
- Country = country name
- Latitude= latitude
- Longitude = longitude

The third dimension table will contain US demographics data. The columns below will be extracted from the us demographics dataframe:
- City = city name
- State = state name
- Total Population = total city population 
- Race = selection of White, Hispanic or Latino, Asian, Black or African-American, American Indian and Alaska Native
- Count = Number Count based on respective Race selection

The fact table will contain information from the I94 immigration data joined with the city temperature data and the demographics data on i94port:
- i94yr = 4 digit year
- i94mon = numeric month
- i94res = 3 digit code of residence country
- i94cit = 3 digit code of birth country
- i94port = 3 character code of destination city
- arrdate = arrival date
- destination_city = full name of city
- depdate = departure date
- i94visa = reason for immigration
- hispanic_ratio = percentage of hispanic people in the city
- AverageTemperature = average temperature of destination city


### Step 5: Complete Project Write Up

* Clearly state the rationale for the choice of tools and technologies for the project.
    * Spark can be better used in batch processing, compared with other data loading methods (flink is more suitable for real-time flow processing),Spark has an advantage in dealing with large files. After grouping the data to a smaller size, Pandas was then used to analyze the data further. 
* Propose how often the data should be updated and why.
    * The immigration data should be updated yearly (or at the same frequency that the right authority releases the data)
* Write a description of how you would approach the problem differently under the following scenarios:
    * The data was increased by 100x.
        * If the increase is more than 100 times, Spark cannot be executed on a local device. Spark cluster on Hadoop is needed to meet the data needs.  After grouping the data, the analysis could be continued with Pandas, the same way that it has been done here.
    * The data populates a dashboard that must be updated on a daily basis by 7am every day.
        * If the data needed to populate a dashboard on a daily basis, it would help to use a workflow orchestration tool, such as Apache Airflow, Luigi, etc to schedule and execute the task. It would also help if the dashboard was connected to a real-time database.  	
    * The database needed to be accessed by 100+ people.
        * From the current situation, my project list may not have as many visitors, but in terms of access, it could be handled by any database. But if this number becomes too large, then perhaps hosting it on a cloud provider, such as AWS, would help with the load.
